# Проект "Исследование надёжности заёмщиков"

## План проекта
**Задача исследования**: выявление влияния семейного положения и количества детей клиента на факт погашения кредита в срок

**Ход исследования**:
- Изучение данных
- Пердобработка данных
- Исследование данных на предмет зависимостей

**Зависимости которые планируется проверить**:
-  между количеством детей и возвратом кредита в срок
- между семейным положением и возвратом кредита в срок
-  между уровнем дохода и возвратом кредита в срок
- между разными целями кредита и возвратом кредита в срок


## 1. Изучение данных

In [ ]:
import pandas as pd #импортируем библиотеку pandas

In [ ]:
data = pd.read_csv('data.csv') #считываем файл и сохраняем его в датафрейм 

In [ ]:
data.head(20) #выводим первые 20 строк датафрейма на экран

In [ ]:
data.info() #выводим общую информацию о датафрейме

### Промежуточный вывод
Итак, в таблице двенадцать столбцов и 21525 строк. Типы данных в столбцах — `int64`, `float64`, `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## 2. Предобработка данных

### 2.1 Удаление пропусков

In [ ]:
data.isna().sum() #выводим количество пропущенных значений для каждого столбца

In [ ]:
for item in data['income_type'].unique():
    data.loc[(data['income_type'] == item) & (data['total_income'].isna()), 'total_income'] = data.loc[data['income_type'] == item, 'total_income'].median()
#заменяем пропуски в столбце 'total_income' медианными значениями по типу занятости

### 2.2 Обработка аномальных значений

In [ ]:
for item in data['days_employed'].unique():
    data.loc[(data['days_employed'] == item) & (item < 0), 'days_employed'] = abs(item)
#обрабатываем возможные отрицательные значения в столбце 'days_employed'

In [ ]:
data.groupby('income_type')['days_employed'].median() 
#выводим медианное значение трудового стажа в днях для каждого типа занятости

In [ ]:
data['children'].unique() #выводим уникальные значения столбца 'children'

Получаем `[1, 0, 3, 2, -1, 4, 20, 5]` и видим два аномальных значения - -1 и 20.

In [ ]:
data = data[(data['children'] >= 0) & (data['children'] < 20)] #удаляем аномальные значения

In [ ]:
data['children'].unique() #проверяем что аномальных значений больше нет

### 2.3 Удаление пропусков (продолжение)

In [ ]:
for item in data['income_type'].unique():
    data.loc[(data['income_type'] == item) & (data['days_employed'].isna()), 'days_employed'] = data.loc[data['income_type'] == item, 'days_employed'].median()
#заполняем пропуски в столбце 'days_employed' медианными значениями по каждому типу занятости

In [ ]:
data['days_employed'].isna().sum() #проверяем что все пропуски заполнены

### 2.4 Изменение типов данных

In [ ]:
data['total_income'] = data['total_income'].astype('int') 
#заменяем вещественный тип данных в столбце 'total_income' на целочисленный

### 2.5 Обработка дубликатов

In [ ]:
data['education'] = data['education'].str.lower() #обрабатываем неявные дубликаты в столбце 'education'

In [ ]:
data.duplicated().sum() #выводим на экран количество строк-дубликатов

In [ ]:
data = data.drop_duplicates() #удаляем найденные дубликаты

### 2.6 Категоризация данных

In [ ]:
def categorize_income(num): #создаём функцию для категоризации ежемесячного дохода
    if num < 30000:
        return 'E'
    elif num < 50000:
        return 'D'
    elif num < 200000:
        return 'C'
    elif num < 1000000:
        return 'B'
    else:
        return 'A'

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income) #добавляем в датафрейм столбец с категорией дохода

In [ ]:
data['purpose'].unique() #выводим перечень уникальных целей взятия кредита 

In [ ]:
def categorize_purpose(purpose): #создаём функцию для категоризации целей взятия кредита
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвижи' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    else:
        return 'получение образования'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose) #добавляем в датафрейм столбец с категорией целей взятия кредита

### Промежуточный вывод
В ходе этого шага исследования мы выполнили следующее:
- удалили пропуски в данных
- обработали аномальные значения
- изменили типы данных где было необходимо
- обработали дубликаты
- категоризовали данные

## 3. Исследование данных

1. Проверка наличия зависимости между количеством детей и возвратом кредита в срок

In [ ]:
display(data.groupby('children')['debt'].agg(['count', 'sum', 'mean']))

**Вывод:** 
Наибольшее количество должников среди заёмщиков с четырьмя детьми. Следом за ними родители одного или двух детей. Удивительно что среди родителей пятерых детей должников нет, но тут, предположительно, дело в том что таких заёмщиков очень мало для построения статистики. Наименьшее количество должников среди заёмщиков без детей.

2. Проверка наличия зависимости между семейным положением и возвратом кредита в срок

In [ ]:
display(data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']))

**Вывод:** 
Наибольшее количество должников приходится на тех заёмщиков, которые никогда не были в браке (не женатые / не замужние, состоящие в гражданском браке), а среди тех кто был или находится в браке должников меньше (по убывающей - женатые / замужние, в разводе, вдовы / вдовцы).

3. Проверка наличия зависимости между уровнем дохода и возвратом кредита в срок

In [ ]:
display(data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']))

**Вывод:** 
Наибольшее количество должников приходится на заёмщиков с доходами до 200000, а наименьшее - на заёмщиков с доходом от 200000, из чего можно сделать вывод, что чем выше доход - тем надёжнее заёмщик.

4. Проверка наличия зависимости между целями кредита и возвратом кредита в срок

In [ ]:
display(data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']))

**Вывод:** 
Наибольшее количество должников приходится на клиентов, запрашивающих кредит на операции с автомобилем и получение образования, следом за ними - те, кто берут кредит на проведение свадьбы. А самыми добросовестными заёмщиками являются те клиенты, которым нужен кредит на операции с недвижимостью.

5. Исследование возможных причин пропусков в данных

Пропуски в данных бывают двух видов - технологические и ошибки, вызванные человеческим фактором. К технологическим относятся ошибки, возникшие при копировании, записи, транспортировке и т.д. файла. К ошибкам, вызванным человеческим фактором - опечатки, пропуски по невнимательности, случайное удаление данных и т.д.

6. Исследование, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных

Количественные переменные очень часто сравниваются для анализа данных, а значит наличие пустых значений, нулей и т.д. очень сильно повлияет на анализ и будут получены некорректные выводы. Для этой проблемы может быть два решения - заполнить пропуски средним арифметическим значением или медианным значением. Однако при поиске среднего будут учитываться краевые значения, которые могут значительно отличаться от большинства значений, будучи пиковыми. Поэтому заполняя пропуски медианным значением мы не меняем общую статистику значительно, а оставляем её примерно на среднем уровне.

## 4. Общий вывод
**Задачей** исследования было выявление влияния семейного положения и количества детей клиента на факт погашения кредита в срок.
**Входными данными** была статистика о платежеспособности клиентов
Во входных данных были **пропуски** в столбцах `days_employed` и `total_income`, которые мы заполнили соответствующими медианными значениями. Также в столбце `days_employed` были найдены аномальные, а именно отрицательные, данные, которые были заменены на свои модули. В столбце `education` дубликаты были обработаны переводом всех значений в нижний регистр. Полностью дублированные строки были удалены.
Для **категоризации данных** были созданы два новых столбца - `purpose_category` и `total_income_category`, в которых для каждого заёмщика были определены категории целей получения кредита ('операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования') и категории дохода (0–30000 — 'E', 30001–50000 — 'D', 50001–200000 — 'C', 200001–1000000 — 'B', 1000001 и выше — 'A').
В процессе исследования были выявлены следующие **зависимости**:
* Наибольшее количество должников приходится на тех заёмщиков, которые никогда не были в браке (не женатые / не замужние - для них отношение должников к общему числу заёмщиков равно **0.09763948497854077**, состоящие в гражданском браке - **0.09313014029995162**), а среди тех кто был или находится в браке должников меньше (по убывающей - женатые / замужние - **0.07560557866405676**, в разводе - **0.07064760302775441**, вдовы / вдовцы - **0.06624605678233439**).
* Наибольшее количество должников среди заёмщиков с четырьмя детьми - отношение должников к общему числу заёмщиков равно **0.0975609756097561**. Следом за ними родители одного - **0.09234608985024959** или двух - **0.09454191033138401** детей. Удивительно что среди родителей пятерых детей должников нет, но тут, предположительно, дело в том что таких заёмщиков очень мало для построения статистики. Наименьшее количество должников среди заёмщиков без детей.
* Наибольшее количество должников приходится на клиентов, запрашивающих кредит на операции с автомобилем - отношение должников к общему числу заёмщиков равно **0.09347978499649451** и получение образования - **0.09252758274824474**, следом за ними - те, кто берут кредит на проведение свадьбы - **0.07911802853437094**. А самыми добросовестными заёмщиками являются те клиенты, которым нужен кредит на операции с недвижимостью - **0.07255139056831923**.